# Análisis mercado de inversiones Chile (Mayo 2021) 

Este análisis usa datos obtenidos mediante el [API de Fintual](https://fintual.cl/api-docs/index.html) y [un simple script](https://replit.com/@continuumhq/fetch-chile-investment-market-data#main.py) que vuelca en un archivo CSV la "foto" del mercado en el día acual. Los datos usados son la salida de dicho de script.

Lo primero que haremos será tomar solo un subconjunto de esos datos y darles mejores nombres a las columnas:

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)

usecols = [
    'asset_provider_attributes_name', # Company behind the fund   
    'real_asset_attributes_name', # Fund name
    'real_asset_attributes_serie', # Fund series
    'real_asset_attributes_last_day_total_net_assets', # Series' AUM
    'real_asset_attributes_last_day_fixed_fee', # % fixed fee (daily) 
    #
    # Note we will not consider variable fees nor expenses on this analysis
    #
    'real_asset_attributes_last_day_institutional_investors', # Number of institutional investors
    'real_asset_attributes_last_day_shareholders', # Number of investors
    'real_asset_attributes_end_date', # useful to exclude inactive funds
    'conceptual_asset_attributes_currency', # useful to exclude non-CLP funds    
    'conceptual_asset_attributes_category', # useful to include only `mutual_fund`        
]

names = dict(
    asset_provider_attributes_name='financial_institution',
    real_asset_attributes_name='fund_name',
    real_asset_attributes_serie='fund_serie',
    real_asset_attributes_last_day_total_net_assets='serie_aum',
    real_asset_attributes_last_day_fixed_fee='serie_daily_fixed_fee',
    real_asset_attributes_last_day_institutional_investors='serie_n_institutional_investors',
    real_asset_attributes_last_day_shareholders='serie_n_shareholders',
    real_asset_attributes_end_date='fund_end_date',
    conceptual_asset_attributes_currency='fund_currency',
    conceptual_asset_attributes_category='fund_category')

data = pd.read_csv(
    '../input/chile-investment-market-data-may-2021/data.csv', 
    usecols=usecols)[usecols]
data.rename(columns=names, inplace=True)
data

Vamos a agregar algunas columnas adicionales para hacernos la vida mas fácil en el análisis. También agregaremos dos columnas mas "opinionadas":

- is_expensive será `True` si el fee anual de un fondo es mayor que el 1.5%
- is_retail será `True` si el saldo promedio de los aportantes es menor o igual a 100 millones de pesos chilenos.

In [ ]:
# Approximate exchange rate at the end of May 2021:
clp_per_usd = 730
clp_per_usd_M = clp_per_usd * 1000000 

# It's easier to reason about millions of USD than CLPs with too many zeroes:
data['serie_aum_usd_M'] = data['serie_aum'] / clp_per_usd_M

# We commonly speak about annual fees, but they come as daily fees on the data:
data['serie_annual_fixed_fee'] = data['serie_daily_fixed_fee'] * 365

# We can extrapolate the annual incom from the AUM and the annual fee. 
# It's not going to be accurate (AUM will fluctuate, you'll get outflows and inflows, etc).
# But again, it's easier to reason about annuual figures
data['serie_extrapolated_annual_income'] = data['serie_annual_fixed_fee'] * data['serie_aum']
# ...and in millions of USD;
data['serie_extrapolated_annual_income_usd_M'] = data['serie_extrapolated_annual_income'] / clp_per_usd_M

# Now some more opinionated attributes:

# We'll consider anything above 1.5% as an "expensive" fund. 
# We are not alone on this rule of thumb:
# https://www.investopedia.com/ask/answers/032715/when-expense-ratio-considered-high-and-when-it-considered-low.asp
# You might argue that we shouldtweak this to include IVA and set it to ~1.8%.
# But all the players in the industry which are going for lower cost funds
# (Fintual, Focus, Clever by BIECE, even Itau!) are aiming for 1.2% and 
# less. 
#
# So, we consider that 1.5% (including IVA) sounds like a good threshold:
expensive_threshold = 1.5/100 
data['is_expensive'] = data.serie_annual_fixed_fee > expensive_threshold

# Then we'll consider any fund with average shareholder balance below CLP 100M as "retail"
# Surprisingly, there are funds with high balances paying expensive fees. We'll narrow down
# the analysis to the "retail" market and the CLP 100M cutoff point felt reasonable.
data['serie_average_shareholder_balance_clp_M'] = data['serie_aum'] / data['serie_n_shareholders'] / 1000000
retail_threshold = 100 
data['is_retail'] = data.serie_average_shareholder_balance_clp_M <= retail_threshold
data

De las 8k+ filas que vienen ahí, nos quedaremos solo con la data que corresponde a fondos mutuos vigentes denominados en pesos chilenos y que cobran fee de administración fijo mayor a 0.

In [ ]:
series = data[
    # Only mutual funds
    (data.fund_category == 'mutual_fund') 
    # Only CLP (or null, which we'll assume is CLP)
    & (
        (data.fund_currency == 'CLP') |
        (data.fund_currency.isnull())
    )  
    # Only active funds
    & (data.fund_end_date.isnull())
    # Only funds series that charge fixed fees
    & (data.serie_daily_fixed_fee > 0)
]
series


Eso nos deja con ~2k registros para trabajar (es posible que estemos dejando datos relevantes fuera, por lo que los resultados del análisis podrían variar si cambiamos la forma de seleccionar las series/fondos relevantes).

Veamos que tan grande es la torta con la que nos quedamos en términos de AUM (assets under management). Los números en pesos son muy grandes, así que los llevaremos a millones de dólares:

In [ ]:
series.serie_aum_usd_M.sum()

Son mas de 45M de dólares. Probablemente estamos dejando algo de la torta fuera (el [2020 habían 60M+ dólares en la industria según la CMF](https://www.cmfchile.cl/portal/estadisticas/617/w3-article-9673.html)), pero puede tener que ver con excluir los fondos en dólares y/o con algún vacío en los datos originales.

Para simplificarnos la vida, llevemos a millones de dólares el AUM de cada serie. Y con ese dato, miremos como andan los actores de la industria:

In [ ]:
aum_per_institution = pd.pivot_table(series, index='financial_institution', values='serie_aum_usd_M', aggfunc=np.sum, margins=True)
aum_per_institution

En líneas generales tiene sentido. Lo que uno esperaría si la data es razonable [(Banchile, Santander, Bci como las mas grandes)](https://www.cmfchile.cl/portal/estadisticas/617/w3-article-9675.html) es lo que vemos acá.

También podemos describir en líneas generales los datos para mirar si se ven razonables:

In [ ]:
series.describe()

Algo raro hay en `serie_annual_fixed_fee` donde nos saltó un máximo de ~300%. Seguro algo viene mal en la data donde algún número venía malo en la fuente original. Asumamos que ningún fondo debiera cobrar más del 10% de fee. Veamos cuantos casos tenemos arriba de ese 10% en todo el dataset original:

In [ ]:
data[data.serie_annual_fixed_fee >= 10/100] 

Es solo un caso (en el dataset original donde hicimos el análisis). Vamos a excluirlo de las series a analizar y ver como queda el resultado:

In [ ]:
# We are getting data with 300% anual fixed fee, which is obviously wrong. Let's remove anything over 10%
series = series[series.serie_annual_fixed_fee <= 10/100] 
series.describe()

Ahora el fondo más caro es de un ~5.65%. No sabemos si llamarlo razonable, pero está dentro de los valores posibles.

Con más confianza en que los datos están correctos, pasemos a mirar como se reparte la torta de *ingresos por fees* que son la multiplicación del saldo administrado (que ya vimos) por los fees anuales:

In [ ]:
pd.pivot_table(series, index=('financial_institution'), values='serie_extrapolated_annual_income_usd_M', aggfunc=np.sum, margins=True)

Suena razonable. Tenemos nuevamente a Banchile, Santander y Bci en el top 3. (Hay números sospechosamente pequeños en Ameris y Toesca, pero no debiera cambiar nada significativo en el grueso del análisis).

Lo relevante es que tenemos una industria de más de 550M de dólares de ingresos anuales por fees. 

Ahora veremos como se segmenta esta industria en las categorías de fondos caros y retail.

Partamos por algunos fondos "caros":

In [ ]:
expensive_series = series[series.is_expensive]
expensive_series



Gran parte de estos fondos "caros" parecen cobrar alrededor de ~2%, ¡pero sabemos que hay casos que llegan al 5%! Acá se puede ver la distribución:

In [ ]:
expensive_series.serie_annual_fixed_fee.hist()

En cuanto a los fondos "baratos", esta es la lista exploratoria:

In [ ]:
inexpensive_series = series[~series.is_expensive]
inexpensive_series

¿Cuanta plata hay metida en fondos "baratos"? ¿Cuantoas en fondos "caros"? (en millones de dólares)

In [ ]:
aum_per_institution_and_is_expensive = pd.pivot_table(series, columns='is_expensive', index='financial_institution', values='serie_aum_usd_M', aggfunc=np.sum, fill_value=0, margins=True)
aum_per_institution_and_is_expensive

Uhm. O sea, ¿Qué porcentaje del saldo administrado está en fondos caros?

In [ ]:
aum_per_institution_and_is_expensive[True]['All'] / aum_per_institution_and_is_expensive['All']['All']

¡Como un tercio!



Veamos ahora los ingresos que obtienen las administradoras:

In [ ]:
income_per_institution_and_is_expensive = pd.pivot_table(series, columns='is_expensive', index='financial_institution', values='serie_extrapolated_annual_income_usd_M', aggfunc=np.sum, fill_value=0, margins=True)
income_per_institution_and_is_expensive

Oh, ¿entonces qué porcentaje de los ingresos provienen de fondos "caros"?

In [ ]:
income_per_institution_and_is_expensive[True]['All'] / income_per_institution_and_is_expensive['All']['All']

¡Dos tercios!

Acotemos ahora el análisis a los fondos "retail" que es donde se puso interesante la competencia recientemente. Porque hay algunos fondos que son sorprendentemente caros a pesar de tener aportantes con saldos altos (¡incluso institucionales!), pero no son tan relevantes para la competencia retail de inversiones. Veamos, que fondos quedan en esta categoría retail:

In [ ]:
retail_series = series[series.is_retail]
retail_series

Todo luce razonable: series con saldos bajos o de algunos millones de pesos. En el histograma se ve mas claro:

In [ ]:
retail_series.serie_average_shareholder_balance_clp_M.hist()

Hagamos un análisis similar al anterior: ¿Qué proporción de los saldos en fondos mutuos serían "retail"? ¿Y que proporción de los ingresos?

In [ ]:
aum_per_institution_and_is_retail = pd.pivot_table(series, columns='is_retail', index='financial_institution', values='serie_aum_usd_M', aggfunc=np.sum, fill_value=0, margins=True)
aum_per_institution_and_is_retail

In [ ]:
aum_per_institution_and_is_retail[True]['All'] / aum_per_institution_and_is_retail['All']['All']

Casi dos tercios de los fondos analizados estarían en las series catalogadas como retail

In [ ]:
income_per_institution_and_is_retail = pd.pivot_table(series, columns='is_retail', index='financial_institution', values='serie_extrapolated_annual_income_usd_M', aggfunc=np.sum, fill_value=0, margins=True)
income_per_institution_and_is_retail

In [ ]:
income_per_institution_and_is_retail[True]['All'] / income_per_institution_and_is_retail['All']['All']

¡Y casi 4/5 del los ingresos de la industria provienen de las series retail!

En conclusión, suena a que tanto la categoría de "retail" como de "caro" sobrepondera ingresos para la industria.

¿Qué pasaría entonces si el cliente retail huye de los fondos caros? ¿Cuanta plata podría moverse?

In [ ]:
series[series.is_expensive & series.is_retail].serie_aum_usd_M.sum()

¡Más de 14 mil millones de dolares!

Acá podemos ver el desglose de los montos ahorrados/invertidos si son o no retail y si son o no "caros"

In [ ]:
pd.pivot_table(series, columns=('is_retail', 'is_expensive'), index='financial_institution', values='serie_aum_usd_M', aggfunc=np.sum, fill_value=0, margins=True)

Finalmente, el desglose de *ingresos* que obtiene cada administradora segun si las series son retail o no y si son "caras" o no:

In [ ]:
income_per_institution_and_is_retail_and_is_expensive = pd.pivot_table(series, columns=('is_retail', 'is_expensive'), index='financial_institution', values='serie_extrapolated_annual_income_usd_M', aggfunc=np.sum, fill_value=0, margins=True)
income_per_institution_and_is_retail_and_is_expensive

¿Qué porcentaje de ingresos de la industria está en juego entonces?

In [ ]:
income_per_institution_and_is_retail_and_is_expensive[(True, True)]['All'] / income_per_institution_and_is_retail_and_is_expensive['All']['All'] 

¡Harto más que la mitad!